# iEEG Source‑Estimation Pipeline

Walk‑through with **OpenNeuro dataset `ds003848` (subject `sub‑RESP0521`)**.

This notebook demonstrates **each function** in the pipeline script you are working on. Run the cells sequentially; every step is self‑contained and heavily commented to make the logic clear.

## 0. Set‑up

*If you are running this on Colab or a fresh environment, uncomment the next cell to install dependencies*.

In [1]:
# !pip install --quiet mne nibabel nilearn openneuro-py pandas numpy

In [2]:
import os
import mne
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from nilearn import plotting
from openneuro import download

from esi import (
    PipelineConfig,
    load_raw,
    load_electrodes,
    set_montage_and_types,
    make_subject_id,
    build_forward_model,
    create_evoked_or_epochs,
    estimate_sources,
    save_nifti,
)

print("MNE version:", mne.__version__)


/home/sms/.pyenv/versions/nrimg/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MNE version: 1.10.0


## 1. Download sample data from OpenNeuro

We will grab **one session** (`ses‑1`) of `sub‑RESP0521`.
Download only the iEEG recording (`.vhdr/.eeg/.vmrk`), the electrodes table, and the T1‑weighted MRI.

If you already have these files locally, point `dataset_dir` to that directory and skip the download.

In [3]:
dataset_dir = Path("/home/sms/Github/datasets/ieeg")
root_dir = Path("/home/sms/Github/epimage/eeg")

dataset_dir.mkdir(parents=True, exist_ok=True)
os.getcwd()

'/home/sms/Github/epimage/eeg'

In [4]:
dataset_id = "ds003848"
sub = "sub-RESP0521"
ses = "ses-1"

# # Download (Method 1)
# download(
#     dataset=dataset_id,
#     target_dir=dataset_dir,
#     include=[
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.vhdr",
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.eeg",
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.vmrk",
#         "sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.tsv",
#         "sub-RESP0521/anat/sub-RESP0521_T1w.nii.gz",
#     ]
# )

# # Download (Method 2)
# %cd $dataset_dir
# ! openneuro-py download --dataset=$dataset_id --include=sub-RESP0521
# %cd $root_dir


### Define paths

In [5]:
subjects_dir = dataset_dir / dataset_id # / sub
ieeg_file = next((subjects_dir / sub / ses / "ieeg").glob("*_ieeg.vhdr"))
electrodes_tsv = subjects_dir / sub / ses / "ieeg" / f"{sub}_{ses}_electrodes.tsv"
mri_path = next((subjects_dir / sub / ses / "anat").glob("*_T1w.nii"))

print("iEEG file :", ieeg_file)
print("Electrodes:", electrodes_tsv)
print("T1 MRI    :", mri_path)

mne.utils.set_config("SUBJECTS_DIR", subjects_dir, set_env=True)


iEEG file : /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.vhdr
Electrodes: /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.tsv
T1 MRI    : /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/anat/sub-RESP0521_ses-1_rec-deface_T1w.nii


## 2. Prepare electrodes CSV (x,y,z,name,type)
The pipeline expects **comma‑separated** values with at least `x,y,z`.

In [6]:
csv_path = electrodes_tsv.with_suffix('.csv')
df = pd.read_csv(electrodes_tsv, sep='\t')
# Keep only good rows where x/y/z are finite
df_valid = df[np.isfinite(df['x']) & np.isfinite(df['y']) & np.isfinite(df['z'])]
cols = ['name', 'x', 'y', 'z']
if 'hemisphere' in df_valid.columns:
    cols.append('hemisphere')  # example extra col
df_valid[cols].to_csv(csv_path, index=False)
print("Saved CSV to", csv_path)

Saved CSV to /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.csv


## 3. Create a `PipelineConfig`

In [7]:
cfg = PipelineConfig(
    ieeg_path=ieeg_file,
    electrodes_path=csv_path,
    mri_path=mri_path,
    subjects_dir=subjects_dir,      # fall back to fsaverage + spherical head‑model
    spacing_mm=6.0,
    inverse_method="dSPM",
    snr=3.0,
    time_window=(0, 30),        # first 30 s as example
    coord_units="mm",           # TSV provides mm
    keep_time=False,
    to_mni=True,                # morph to fsaverage/MNI
    verbose=False,
)
cfg

PipelineConfig(ieeg_path=PosixPath('/home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.vhdr'), electrodes_path=PosixPath('/home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.csv'), mri_path=PosixPath('/home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/anat/sub-RESP0521_ses-1_rec-deface_T1w.nii'), subjects_dir=PosixPath('/home/sms/Github/datasets/ieeg/ds003848'), spacing_mm=6.0, inverse_method='dSPM', snr=3.0, time_window=(0, 30), loose=0.2, depth=0.8, out_file=PosixPath('ieeg_sources_z.nii.gz'), verbose=False, coord_units='mm', keep_time=False, to_mni=True)

### 3.1 `load_raw`

In [8]:
raw = load_raw(cfg)
print(raw)
print("Data shape:", raw.get_data().shape)

2025-07-18 20:13:11,062 — INFO — Loading raw data → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.vhdr


<RawBrainVision | sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.eeg, 133 x 963991 (1882.8 s), ~978.3 MiB, data loaded>
Data shape: (133, 963991)


In [9]:
raw

<RawBrainVision | sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.eeg, 133 x 963991 (1882.8 s), ~978.3 MiB, data loaded>

In [10]:
# raw.set_channel_types({name: "seeg" or "ecog"})
# raw.pick_types(eeg=True)  

### 3.2 `load_electrodes`

In [11]:
coords, names, types = load_electrodes(cfg, n_channels=len(df_valid))
print("coords shape:", coords.shape)
print("first 5 coords (m):\n", coords[:5])
print("sample names:", names[:5])
print("types counts:", pd.Series(types).value_counts().to_dict())

2025-07-18 20:13:14,457 — INFO — Reading electrode coordinates → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.csv


coords shape: (64, 3)
first 5 coords (m):
 [[-0.03486902 -0.04051788  0.03658449]
 [-0.03786902 -0.03451788  0.04358449]
 [-0.04186902 -0.02751788  0.04958449]
 [-0.04586902 -0.02051788  0.05858449]
 [-0.04886902 -0.01351788  0.06558449]]
sample names: ['C01', 'C02', 'C03', 'C04', 'C05']
types counts: {'seeg': 64}


### 3.3 `set_montage_and_types`

In [12]:
set_montage_and_types(raw, coords, names, types)
print(">> Montage set. Dig points:", len(raw.info['dig']))

mapping = {name: "eeg" for name in names}
raw.set_channel_types(mapping)

raw.pick_channels(names, ordered=True)


>> Montage set. Dig points: 67
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawBrainVision | sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.eeg, 64 x 963991 (1882.8 s), ~470.8 MiB, data loaded>

### 3.4 `make_subject_id` & `build_forward_model`

In [13]:
subject = make_subject_id(cfg)
fwd, src = build_forward_model(cfg, raw, subject)
print(f"Subject inferred: {subject}")
print(f"Forward solution with {fwd['nsource']} sources and {fwd['nchan']} channels.")

2025-07-18 20:13:15,186 — INFO — Converted sub-RESP0521_ses-1_rec-deface_T1w.nii → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/mri/T1.mgz
2025-07-18 20:13:15,188 — INFO — Setting up 6.0 mm volumetric source space


/home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/mri/T1.mgz


2025-07-18 20:13:18,118 — INFO — Attempting BEM model/solution (requires FreeSurfer surfaces)
2025-07-18 20:13:18,120 — WARNING — BEM surfaces not found → falling back to spherical model


Fitted sphere radius:         55.0 mm
Origin head coordinates:      -7.2 10.6 41.4 mm
Origin device coordinates:    -7.2 10.6 41.4 mm

Equiv. model fitting -> RV = 0.00347455 %%
mu1 = 0.944856    lambda1 = 0.136823
mu2 = 0.667779    lambda2 = 0.683693
mu3 = -0.294888    lambda3 = -0.0101468
Set up EEG sphere model with scalp radius    55.0 mm



2025-07-18 20:13:18,246 — INFO — Computing forward solution (seeg+ecog enabled)


Subject inferred: sub-RESP0521
Forward solution with 13997 sources and 64 channels.


### 3.5 `create_evoked_or_epochs`

In [14]:
data_obj = create_evoked_or_epochs(raw, cfg)
print(type(data_obj), data_obj)

2025-07-18 20:13:18,907 — INFO — Cropping raw data to 0.000–30.000 s


<class 'mne.evoked.EvokedArray'> <Evoked | 'iEEG‑mean' (average, N=1), 0 – 0 s, baseline off, 64 ch, ~80 KiB>


### 3.6 `estimate_sources` (full pipeline wrapper)

In [15]:
zmap, affine = estimate_sources(cfg)
print("Z‑map shape:", zmap.shape)

2025-07-18 20:13:19,084 — INFO — Loading raw data → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_task-Sleep_run-060307_ieeg.vhdr
2025-07-18 20:13:21,758 — INFO — Reading electrode coordinates → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.csv


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


2025-07-18 20:13:22,589 — INFO — Converted sub-RESP0521_ses-1_rec-deface_T1w.nii → /home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/mri/T1.mgz
2025-07-18 20:13:22,590 — INFO — Setting up 6.0 mm volumetric source space


/home/sms/Github/datasets/ieeg/ds003848/sub-RESP0521/mri/T1.mgz


2025-07-18 20:13:25,396 — INFO — Attempting BEM model/solution (requires FreeSurfer surfaces)
2025-07-18 20:13:25,398 — WARNING — BEM surfaces not found → falling back to spherical model


Fitted sphere radius:         55.0 mm
Origin head coordinates:      -7.2 10.6 41.4 mm
Origin device coordinates:    -7.2 10.6 41.4 mm

Equiv. model fitting -> RV = 0.00347455 %%
mu1 = 0.944856    lambda1 = 0.136823
mu2 = 0.667779    lambda2 = 0.683693
mu3 = -0.294888    lambda3 = -0.0101468
Set up EEG sphere model with scalp radius    55.0 mm



2025-07-18 20:13:25,514 — INFO — Computing forward solution (seeg+ecog enabled)
2025-07-18 20:13:26,164 — INFO — Cropping raw data to 0.000–30.000 s


Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


2025-07-18 20:13:26,321 — INFO — Creating diagonal noise covariance
/home/sms/.pyenv/versions/nrimg/lib/python3.12/site-packages/mne/forward/forward.py:1466: RuntimeWarning: divide by zero encountered in divide
  w = 1.0 / d
2025-07-18 20:13:27,431 — INFO — Applying inverse solution (dSPM, λ²=0.111)
/home/sms/.pyenv/versions/nrimg/lib/python3.12/site-packages/mne/minimum_norm/inverse.py:756: RuntimeWarning: divide by zero encountered in divide
  inv["noisenorm"] = 1.0 / np.abs(noise_norm)
/home/sms/.pyenv/versions/nrimg/lib/python3.12/site-packages/mne/minimum_norm/inverse.py:1112: RuntimeWarning: invalid value encountered in multiply
  sol *= noise_norm
2025-07-18 20:13:27,586 — INFO — Morphing volumetric STC to fsaverage (MNI)
2025-07-18 20:13:34,717 — INFO — Creating scale space from the moving image. Levels: 3. Sigma factor: 0.200000.
2025-07-18 20:13:34,724 — INFO — Creating scale space from the static image. Levels: 3. Sigma factor: 0.200000.
2025-07-18 20:13:34,730 — INFO — Opti

ValueError: stc.subject does not match src subject (fsaverage != sub-RESP0521)

### 3.7 `save_nifti` & quick visualisation

In [ ]:
nii_path = Path("ieeg_sources_resp0521_z.nii.gz")
save_nifti(zmap, affine, nii_path)
print("Saved to", nii_path)

# Quick interactive view (Nilearn). Commented for headless servers.
# plotting.view_img(nii_path, threshold=3).open_in_browser()


## 4. Summary
* We loaded a BIDS‑formatted OpenNeuro iEEG dataset.
* Converted electrodes.tsv to a simple CSV.
* Walked through every helper in the pipeline and produced a volumetric Z‑scored source map in MNI space.*